In [3]:
# @title Default title text
from google.colab import drive
drive.mount('/content/drive')
# Smart_Sorting_Transfer_Learning.ipynb

# 1. Install & setup Kaggle API
!pip install -q kaggle
from google.colab import files
files.upload()  # Upload your kaggle.json file here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 2. Download dataset
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -q
!unzip -q fruits-fresh-and-rotten-for-classification.zip -d data

# 3. Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
import glob, random
from tensorflow.keras.preprocessing import image

# 4. Data prep
IMG_SIZE = (224, 224); BATCH = 32
train_dir = 'data/train'
test_dir = 'data/test'

datagen = ImageDataGenerator(
    rescale=1./255, validation_split=0.1,
    rotation_range=20, width_shift_range=0.1,
    height_shift_range=0.1, horizontal_flip=True
)

train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE,
                                        batch_size=BATCH, class_mode='binary',
                                        subset='training')
val_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE,
                                      batch_size=BATCH, class_mode='binary',
                                      subset='validation')

# 5. Build model
base = ResNet50V2(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE,3))
base.trainable = False
model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 6. Train base model
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

# 7. Plot
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.legend(); plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend(); plt.title('Loss')
plt.show()

# 8. Fine-tuning
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False
model.compile(optimizer=optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
fine_hist = model.fit(train_gen, validation_data=val_gen, epochs=5)

# 9. Evaluate
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH, class_mode='binary')
loss, acc = model.evaluate(test_gen)
print(f"Test accuracy: {acc*100:.2f}%")

# 10. Predict sample
sample = random.choice(glob.glob(test_dir + '/*/*.jpg'))
img = image.load_img(sample, target_size=IMG_SIZE)
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0][0]
label = 'Fresh' if pred > 0.5 else 'Rotten'
plt.imshow(img); plt.title(f"{label} ({pred:.2f})"); plt.axis('off')


MessageError: Error: credential propagation was unsuccessful